In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as spst
import scipy.integrate as spint
from scipy.optimize import newton
import matplotlib.pyplot as plt
import warnings

#import sys
#sys.path.insert(sys.path.index('')+1, 'D:/Github/PyFeng')
import pyfeng as pf

# Basket Option

In [3]:
sigma = 0.4*np.ones(4)
spot = np.ones(4)*100
texp=5

In [4]:
m = pf.BsmNdMc(sigma, cor=0.5, rn_seed=1234)
m.simulate(tobs = [texp], n_path=10000)
print(m.n_path)
m.path.shape

10000


(1, 10000, 4)

In [5]:
m.path

array([[[0.15969089, 0.34383065, 0.57141668, 0.42421223],
        [2.81374194, 1.3068322 , 0.78634205, 1.05920794],
        [1.45143979, 9.41924528, 0.71067299, 3.11737835],
        ...,
        [0.34010899, 0.09033437, 0.10311937, 0.584682  ],
        [1.46434278, 1.27087487, 0.61799902, 3.18693079],
        [0.30684685, 0.35355878, 0.72707067, 0.14099113]]])

In [5]:
m.simulate(tobs = [texp], n_path=50000, store=2)
print(m.n_path)
print(m.path.shape)

50000
(1, 50000, 4)


In [8]:
# varying strikes
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)

strikes = np.arange(50, 151, 10)
price = []
for strike in strikes:
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([54.28861405, 47.4587627 , 41.52165148, 36.37763866, 31.89555112,
       28.02031001, 24.67093637, 21.74672612, 19.18599853, 16.95679256,
       15.00046557])

In [7]:
# varying forwards
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)
strike = 100
price = []
for spot in np.arange(50, 151, 10):
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([ 4.6049753 ,  7.87316825, 11.99891419, 16.91824336, 22.53129533,
       28.73942945, 35.47288913, 42.65874825, 50.22366433, 58.11022683,
       66.27677878])

In [8]:
# varying except sigma1=100%
strike = spot = 100
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)
price = []
for sigma1 in np.array([5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100])/100:
    sigma = sigma1 * np.ones(4)
    sigma[0] = 1
    #print(sigma)
    m = pf.BsmNdMc(sigma, cor=0.5, rn_seed=1234)
    m.simulate(tobs = [texp], n_path=400000)
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([18.94495855, 20.45829493, 22.49840945, 24.87785108, 30.11378007,
       35.57949546, 41.05473026, 46.42608331, 51.61861158, 56.59050047,
       65.9322128 ])

# Simpon's method for integration

Let's integrate numerically.

$$ \int_0^T exp(-a*t) dt  = \frac{1}{a}(1 - exp(-a T))$$

In [9]:
a = 10
texp = 2
exact = (1 - np.exp(-a*texp))/a
exact

0.09999999979388464

In [10]:
def f(t, a):
    return np.exp(-a*t)

n_step = 100
tobs = np.arange(0, n_step+1)/n_step * texp
simp = spint.simps(f(tobs, a), dx=texp/n_step)
#simp = spint.simps(f(tobs, a), dx=1) * texp/n_step
print(simp, simp-exact)

0.10000088446767287 8.84673788226209e-07


# For SABR Model
## Integration of sigma(t)

In [5]:
# You can use BsmNdMc because sigma_t is also a geometric Brownian motion

vov = 0.3
texp = 5
m = pf.BsmNdMc(vov, rn_seed=1234)
tobs = np.arange(0, 101)/100*texp

In [6]:
_ = m.simulate(tobs = tobs, n_path=1000)

In [7]:
print(m.path.shape)
sigma_path = np.squeeze(m.path)
print(sigma_path.shape)

(101, 1000, 1)
(101, 1000)


In [8]:
sigma_path

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.00205206, 0.99347145, 1.05514857, ..., 1.05316024, 0.9588047 ,
        1.03828247],
       [1.05074607, 0.94317781, 0.9468966 , ..., 0.89663093, 0.96922105,
        1.02251224],
       ...,
       [2.16851389, 0.29669753, 1.10119963, ..., 0.13990833, 0.62370979,
        1.03155781],
       [2.09349691, 0.30594931, 1.15396032, ..., 0.1311281 , 0.55414525,
        1.15584126],
       [2.42973877, 0.26242663, 1.13376573, ..., 0.12997018, 0.5138661 ,
        1.24084495]])

In [14]:
sigma_final = sigma_path[-1,:]

In [10]:
int_var = spint.simps(sigma_path**2, dx=1, axis=0)/100
int_var.shape

(1000,)

In [11]:
int_var

array([ 1.29802399,  0.73765394,  1.31526303,  0.55124365,  0.63295192,
        1.37330276,  0.40144443,  1.7929296 ,  2.69373586,  0.32141508,
        0.68065474,  1.00584853,  1.13553417,  0.76863878,  1.40566842,
        0.5044663 ,  0.39078481,  2.21799995,  1.31269705,  0.56135306,
        1.56333888,  0.64660353,  1.03679484,  0.82922363,  0.47525061,
        2.71556944,  0.3868775 ,  2.30256972,  0.89661053,  0.80161885,
        0.28933611,  3.77241805,  1.05331416,  0.63456937,  0.5918348 ,
        1.37020097,  1.05734819,  0.66114226,  1.26687819,  0.59061402,
        0.42996024,  3.05406169,  0.30045798,  4.21393842,  0.21261776,
        5.42374707,  0.71077905,  1.06753162,  1.25273576,  0.64102903,
        0.36071037,  1.97904772,  0.75162785,  1.12305064,  0.45261906,
        1.55609758,  1.52643671,  0.53103972,  0.8406454 ,  1.06471408,
        1.27091103,  0.57996366,  0.87964745,  0.81902922,  2.68610776,
        0.32358328,  1.33558855,  0.56005249,  0.8676087 ,  0.79